In [1]:
!git clone https://github.com/partha117/GREEN.git
%cd GREEN
!pip install -e .

Cloning into 'GREEN'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 140 (delta 51), reused 61 (delta 18), pack-reused 0
Receiving objects: 100% (140/140), 65.58 KiB | 10.93 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/GREEN
Obtaining file:///content/GREEN
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached

In [5]:
! pip freeze > requirements.txt

In [ ]:
from google.colab import drive
import pandas as pd
import json
import re
from green_score.green import compute

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_output_df = pd.read_csv('/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/Test_output.csv')
val_output_df = pd.read_csv('/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/Val_output.csv')

In [ ]:
with open("/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/data/annotation_quiz_all.json", "r") as f:
    ground_truth = json.load(f)

In [ ]:
test_ground_truth = ground_truth['test']

In [ ]:
with open("/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/Task1_Result.json", "r") as f:
    val_ground_truth = json.load(f)

In [ ]:
def get_map(data):
  output = dict()
  for item in data:
    if 'report' in item:
      output[item['id']] = item['report']
    else:
      temp = dict()
      for key, value in item.items():
        if key in ['heart', 'bone', 'mediastinal', 'other', 'lung']:
          temp[key] = value
      output[item['id']] = temp
  return output

In [ ]:
val_output_map = get_map(val_ground_truth)
test_output_map = get_map(ground_truth['test'])

In [ ]:
def seperator(text):
  text = text.lower()
  patterns = {
      "lung": re.compile(r"lungs?\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "heart": re.compile(r"hearts?\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|lungs?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "mediastinal": re.compile(r"mediastinal\s*:\s*(.+?)(?=\n(?:bones?|lungs?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "bone": re.compile(r"bones?\s*:\s*(.+?)(?=\n(?:mediastinal|lungs?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "others": re.compile(r"others\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|lungs?|hearts?)\s*:|$)", re.IGNORECASE | re.DOTALL)
  }

  # Extracting the information using regex
  extracted_info = {key: (match.group(1).strip() if (match := pattern.search(text)) else "") for key, pattern in patterns.items()}
  return extracted_info

In [ ]:
test_output_df['formatted_output'] = test_output_df['output'].apply(seperator)
val_output_df['formatted_output'] = val_output_df['output'].apply(seperator)

In [ ]:
val_output_df['ground_truth'] = val_output_df['id'].apply(lambda x: val_output_map[x])
test_output_df['ground_truth'] = test_output_df['id'].apply(lambda x: test_output_map[x])

In [ ]:
def get_region(df, region):
  predicted = []
  ground_truth = []
  for index, row in df.iterrows():
    if region in row['ground_truth'].keys():
      predicted.append(row['formatted_output'][region])
      ground_truth.append(row['ground_truth'][region])
  return predicted, ground_truth

In [ ]:
model_name = "StanfordAIMI/GREEN-radllama2-7b"

In [ ]:
predicted, g_truth = get_region(test_output_df, 'lung')
compute(model_name, g_truth, predicted, output_dir=".")

In [ ]:
predicted, g_truth = get_region(test_output_df, 'bone')
compute(model_name, g_truth, predicted, output_dir=".")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

...

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:29, 89.69s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Summary]: Green average 0.0 and standard variation 0.0 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.4. 
  The candidate report falsely suggests that the bones are not clearly visible. 

 (b) Missing a finding present in the reference: 0.2. 
  The candidate report misses the finding of intact bony structures. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  18.736431074142455


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
predicted, g_truth = get_region(test_output_df, 'heart')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:20, 80.73s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Summary]: Green average 0.4 and standard variation 0.48989794855663565 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 1.0. 
 None 

 (b) Missing a finding present in the reference: 0.6. 
  The candidate report does not mention the normal heart size. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 0.8. 
  The candidate report does not confirm that the heart is normal. 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  16.741691732406615


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
predicted, g_truth = get_region(test_output_df, 'mediastinal')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:27, 87.73s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...
[Summary]: Green average 0.4 and standard variation 0.48989794855663565 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 1.0. 
 None 

 (b) Missing a finding present in the reference: 1.0. 
 None 

 (c) Misidentification of a finding's anatomic location/position: 0.8. 
  The candidate report suggests that the mediastinal region is not clearly visible, which is not mentioned in the reference report. 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  17.833882093429565


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
predicted, g_truth = get_region(test_output_df, 'others')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:26, 86.99s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Summary]: Green average 0.0 and standard variation 0.0 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 1.0. 
 None 

 (b) Missing a finding present in the reference: 0.4. 
  The candidate report does not mention any findings or observations from the radiology examination. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 0.8.  The candidate report does not mention any comparisons to previous studies..
Seconds per example:  17.990904998779296


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# predicted, g_truth = get_region(val_output_df, 'lung')
# compute(model_name, g_truth, predicted, output_dir=".")

In [ ]:
predicted, g_truth = get_region(val_output_df, 'heart')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:40, 100.92s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...
[Summary]: Green average 0.44666666666666666 and standard variation 0.32359096265364257 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 1.0. 
 None 

 (b) Missing a finding present in the reference: 0.2. 
  Absence of pneumothorax, effusion, and acute bony abnormality. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  20.771319913864136


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

In [ ]:
predicted, g_truth = get_region(val_output_df, 'bone')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:24, 84.78s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...
[Summary]: Green average 0.6 and standard variation 0.48989794855663565 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 1.0. 
 None 

 (b) Missing a finding present in the reference: 0.6. 
  Degenerative changes of the spine. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  17.606922149658203


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
predicted, g_truth = get_region(val_output_df, 'mediastinal')
compute(model_name, g_truth, predicted, output_dir=".")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
1it [01:28, 88.96s/it]


==== End Inference ====
Saving generated response to prompt to  ./results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Summary]: Green average 0.6 and standard variation 0.48989794855663565 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.8. 
  Visibility of mediastinal structures 

 (b) Missing a finding present in the reference: 0.6. 
  Absence of large pleural effusion. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  18.741369104385377


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# predicted, g_truth = get_region(val_output_df, 'others')
# compute(model_name, g_truth, predicted, output_dir=".")